# Customizing the Build/Train/Deploy MLOps Project Template

We recently announced [Amazon SageMaker Pipelines](https://aws.amazon.com/sagemaker/pipelines/), the first 
purpose-built, easy-to-use Continuous Integration and Continuous Delivery (CI/CD) service for machine learning. 
SageMaker Pipelines has three main components which improves the operational resilience and reproducibility of your 
workflows: Pipelines, Model Registry, and Projects. 

SageMaker Projects introduce MLOps templates that automatically provision the underlying resources needed to enable 
CI/CD capabilities for your Machine Learning Development Lifecycle (MLDC). Customers can use a number of built-in 
templates or create your own custom templates.

This example will focus on using one of the MLOps templates to bootstrap your ML project and establish a CI/CD 
pattern from seed code. We’ll show how to use the built-in Build/Train/Deploy Project template as a base for a 
customer churn classification example. This base template will enable CI/CD for training machine learning models, 
registering model artifacts to the Model Registry, and automating model deployment with manual approval and automated 
testing.

## MLOps Template for Build, Train, and Deploy

We’ll start by taking a detailed look at what AWS services are launched when this build, train, deploy MLOps template 
is launched. Later, we’ll discuss how the skeleton can be modified for a custom use case. 

To get started with SageMaker Projects, [they must be first enabled in the SageMaker Studio console](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-studio-updates.html). 
This can be done for existing users or while creating new ones:

![enable_projects](https://user-images.githubusercontent.com/18154355/102388848-2c390000-3f87-11eb-91ae-c62eccbd4260.png)

Within Amazon SageMaker Studio, you can now select “Projects” from a drop-down menu on the “Components and registries” 
tab as shown below:

![select_projects](https://user-images.githubusercontent.com/18154355/102389269-b41f0a00-3f87-11eb-8a7b-c68595f550be.png)

From the projects page you’ll have the option to launch a pre-configured SageMaker MLOps template. We'll select the build, train and deploy template:

![create_project](https://user-images.githubusercontent.com/18154355/102389247-aff2ec80-3f87-11eb-8789-33390d217b22.png)
NOTE: Launching this template will kick off a model building pipeline by default and will train a regression model. This will incur a small cost.

Once the project is created from the MLOps template, the following architecture will be deployed:

![deep_dive](https://user-images.githubusercontent.com/18154355/102389254-b1241980-3f87-11eb-8cd5-af495b005ab0.png)



## Modifying the Seed Code for Custom Use Case

After your project has been created the architecture shown above will be deployed and the visualization of the 
Pipeline will be available in the “Pipelines” drop down menu within SageMaker Studio.

In order to modify the seed code from this launched template, we’ll first need to clone the AWS CodeCommit 
repositories to our local SageMaker Studio instance. From the list of projects, select the one that was just 
created. Under the “Repositories” tab you can select the hyperlinks to locally clone the AWS CodeCommit repos:

![clone](https://user-images.githubusercontent.com/18154355/102389244-aec1bf80-3f87-11eb-88a0-7ff1f17d2430.png)


### ModelBuild Repo

The “ModelBuild” repository contains the code for preprocessing, training, and evaluating the model. 
The seed code trains and evaluates a model on the [UCI Abalone dataset](https://archive.ics.uci.edu/ml/datasets/abalone). We can modify these files in order to 
solve our own customer churn use-case.

![model_build](https://user-images.githubusercontent.com/18154355/102390511-57245380-3f89-11eb-973c-6fa9fcdfd65c.png)


We’ll need a dataset accessible to the project. The easiest way to do this is to open a new SageMaker notebook 
inside Studio and run the following cells:

In [ ]:
!wget http://dataminingconsultant.com/DKD2e_data_sets.zip
!unzip -o DKD2e_data_sets.zip
!mv "Data sets" Datasets

In [ ]:
import os
import boto3
import sagemaker
prefix = 'sagemaker/DEMO-xgboost-churn'
region = boto3.Session().region_name
default_bucket = sagemaker.session.Session().default_bucket()
role = sagemaker.get_execution_role()

RawData = boto3.Session().resource('s3')\
.Bucket(default_bucket).Object(os.path.join(prefix, 'data/RawData.csv'))\
.upload_file('./Datasets/churn.txt')

print(os.path.join("s3://",default_bucket, prefix, 'data/RawData.csv'))


We'll need to rename the `abalone` directory to `customer_churn`. That will require us to replace `codebuild-buildspec.yml`
in your Studio project with the one found in [this directory](modelbuild/codebuild-buildspec.yml) 

Next, replace the `preprocess.py`, `evaluate.py` and `pipeline.py` with the ones found in [this example directory](modelbuild/pipelines/customer_churn).

**Note: You'll need to replace the `default_value` of "InputDataURL" with the URL you obtained when uploading the data above.**


## Trigger a new Pipeline Execution through git commit

By committing these changes to the AWS CodeCommit repository (easily done in SageMaker Studio source control tab), a 
new Pipeline execution will be triggered since there is an EventBridge monitoring for commits.  After a few moments, 
we can monitor the execution by selecting your Pipeline inside of the SageMaker Project.

![git_push](https://user-images.githubusercontent.com/18154355/102389260-b2554680-3f87-11eb-870d-70e202bd591e.png)

![execution](https://user-images.githubusercontent.com/18154355/102389259-b2554680-3f87-11eb-833a-7fa1e95696dc.png)

![dag](https://user-images.githubusercontent.com/18154355/102389252-b08b8300-3f87-11eb-8f23-dd1e53ef1b09.png)

Once completed, we can go to our “Model groups” tab inside of the SageMaker Project and inspect the metadata attached 
to the model artifacts. If everything looks good, we can manually approve the model:

<img width="966" alt="model_group" src="https://user-images.githubusercontent.com/18154355/102389264-b3867380-3f87-11eb-9cdb-45dd88bccbc6.png">

![model_status](https://user-images.githubusercontent.com/18154355/102389266-b3867380-3f87-11eb-8465-8763de23ac86.png)

This approval will trigger the ModelDeploy pipeline and expose an endpoint for real time inference.

<img width="306" alt="endpoints" src="https://user-images.githubusercontent.com/18154355/102389257-b2554680-3f87-11eb-8612-bbdb83a22552.png">

## Trigger a new Pipeline Execution through SDK

Alternatively you can also retrieve and execute an existing pipeline through the sagemaker SDK. The template created a 
file `get_pipeline` which you can use to trigger an execution in your own notebook


In [ ]:
# This is the module name or the path to your pipeline.py file.
from pipelines.customerchurn.pipeline import get_pipeline

model_package_group_name = f"CustomerChurnPackageGroup"
pipeline_name = f"CustomerChurnDemo-p-ewf8t7lvhivm"


# These variables were defined the IAM role.
pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
)

### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [ ]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()



In [ ]:
execution.describe()

In [ ]:
execution.wait()


### Parametrized Executions

We can run additional executions of the pipeline specifying different pipeline parameters. The parameters argument is a 
dictionary whose names are the parameter names, and whose values are the primitive values to use as overrides of the defaults.

Of particular note, based on the performance of the model, we may want to kick off another pipeline execution, but this 
time on a compute-optimized instance type and set the model approval status automatically be "Approved". This means 
that the model package version generated by the `RegisterModel` step will automatically be ready for deployment through 
CI/CD pipelines, such as with SageMaker Projects.

In [ ]:
# Note: You can change the ModelApprovalStatus to "PendingManualApproval". This is the default set in the pipeline.py file.

execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.c5.xlarge",
        ModelApprovalStatus="Approved",
    )
)

In [ ]:
execution.wait()

In [ ]:
execution.list_steps()

